# COLLECTE ET HARMONISATION DES DONNEES - REPUBLIQUE DU BENIN

## Objectifs
   - **Thématique**: Démographie - Economie - Santé - Education - Géographie
   - **Sources**: INStad - Portails Open Data - Worl Bank API - Web scraping
   - **Couverture**: Tous les départements du bénin
   - **Période**: 2020 - 2025 (Selon la disponibilité des données au niveau des sources)
### Python version: 3.13.7

### Configuration des imports

In [ ]:
# =============================
# Imports
# =============================

# Standard library
import os
import re
import json
import time
import zipfile
import warnings
import logging
from io import BytesIO
from pathlib import Path
from datetime import datetime
from typing import Optional
from urllib.parse import urljoin, urlparse

# Third-party
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup

# =============================
# Configurations globales
# =============================

# Warnings
warnings.filterwarnings("ignore", category=UserWarning, module="bs4")

# Logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)

# Pandas
pd.set_option("display.max_rows", 100)  # nb max de lignes affichées
pd.set_option("display.max_columns", None)  # affiche toutes les colonnes
pd.set_option("display.float_format", "{:.2f}".format)  # formatage des floats
pd.set_option("display.expand_frame_repr", False)  # évite retour ligne inutile

# Matplotlib / Seaborn
plt.style.use("seaborn-v0_8-whitegrid")  # style moderne et lisible
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12
sns.set_palette("Set2")

# =============================
# Check initialisation
# =============================

logging.info("Librairies et configurations chargées")
logging.info("Début de la collecte de données: %s", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))


### Configuration des dossiers

#### Fonction de configurations de dossiers, réutilisable

In [ ]:
def init_directory(base_dir: Optional[Path]) -> dict:
    """
    Initialise l'aborescence de données dans le projet

    :param base_dir: Optionnel, Répertoire de base du projet, par défaut répertoire où le script est appelé
    :return: Retourne un dictionnaire des chemins créés
    """

    if base_dir is None:
        base_dir = Path(__file__).resolve().parent

    data_dir = base_dir / "data"  # Répertoire ou se trouvera les données
    dirs = {
        "data": data_dir,
        "raw_data": data_dir / "raw_data",  # Répertoire des données brutes
        "processed_data": data_dir / "processed_data",  # Répertoire des données traitées
        "final_data": data_dir / "final_data",  # Répertoire des données finales
    }